In [2]:
import pickle
import random
import time
import math
import os
from datetime import datetime
from typing import List, Tuple

class Keller5ResearchBatch:
    DIMENSION = 5
    TARGET_SIZE = 28
    
    def __init__(self):
        filename = f'keller_codes_d{self.DIMENSION}.data'
        print(f"Loading Keller5 data from {filename}...")
        with open(filename, 'rb') as f:
            self.codes_int = pickle.load(f)
        
        self.n = len(self.codes_int)
        self.best_clique = []
        self.best_size = 0
        self.current_seed = 0
        
        # Create statistics directory immediately
        if not os.path.exists("statistics"):
            os.makedirs("statistics")

    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v: return False
        a, b = (u, v) if u < v else (v, u)
        return ((self.codes_int[a] >> b) & 1) == 0

    def get_all_neighbors(self, v: int) -> List[int]:
        return [u for u in range(self.n) if self.are_neighbors(v, u)]

    def vertex_conflict(self, v: int, vertices: List[int]) -> int:
        return sum(1 for u in vertices if u != v and not self.are_neighbors(v, u))

    def compute_conflict(self, vertices: List[int]) -> int:
        conflicts = 0
        for i in range(len(vertices)):
            for j in range(i + 1, len(vertices)):
                if not self.are_neighbors(vertices[i], vertices[j]):
                    conflicts += 1
        return conflicts

    def improve_with_sa(self, vertices: List[int], iterations: int, initial_temp: float, cooling_rate: float) -> Tuple[List[int], int]:
        current = vertices.copy()
        current_conflict = self.compute_conflict(current)
        best = current.copy()
        best_conflict = current_conflict
        temperature = initial_temp
        stagnant_counter = 0
        last_best_conflict = current_conflict

        for iteration in range(iterations):
            # Stochastic selection with exponential weighting
            v_confs = [self.vertex_conflict(v, current) for v in current]
            weights = [math.exp(c) for c in v_confs]
            worst_idx = random.choices(range(len(current)), weights=weights, k=1)[0]
            
            temp_set = current[:worst_idx] + current[worst_idx+1:]
            
            # Sampling candidates
            sample_size = 500 if len(current) >= 26 else 300
            candidates = random.sample(range(self.n), min(sample_size, self.n))
            
            best_replacement = -1
            min_conflicts = float('inf')
            for v in candidates:
                if v in temp_set: continue
                conf = self.vertex_conflict(v, temp_set)
                if conf < min_conflicts:
                    min_conflicts, best_replacement = conf, v
                if conf == 0: break
            
            if best_replacement == -1: continue
            
            new_vertices = temp_set + [best_replacement]
            new_conflict = self.compute_conflict(new_vertices)
            delta = new_conflict - current_conflict
            
            if delta <= 0 or (temperature > 1e-8 and random.random() < math.exp(-delta / temperature)):
                current, current_conflict = new_vertices, new_conflict
                if current_conflict < best_conflict:
                    best, best_conflict = current.copy(), current_conflict
            
            # Reheating logic
            if best_conflict == last_best_conflict: stagnant_counter += 1
            else: stagnant_counter, last_best_conflict = 0, best_conflict
            
            if stagnant_counter > 1500:
                temperature = max(temperature, initial_temp * 0.4)
                stagnant_counter = 0

            temperature *= cooling_rate
            if best_conflict == 0: break
        return best, best_conflict

    def save_to_statistics(self, clique: List[int], seed: int, duration: float):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        size = len(clique)
        filename = f"statistics/k5_sz{size}_{timestamp}_seed{seed}.txt"
        
        with open(filename, "w", encoding="utf-8") as f:
            f.write(f"KELLER 5 RESEARCH REPORT\nID: {timestamp}\nSeed: {seed}\nTime: {duration:.2f}s\nSize: {size}\n")
            f.write("-" * 50 + "\nVertices: " + str(sorted(clique)) + "\n" + "-" * 50 + "\n")
            c_set = set(clique)
            for i, v in enumerate(sorted(clique)):
                all_neighbors = self.get_all_neighbors(v)
                f.write(f"Vertex {i+1:2d}: ID {v:4d} | Neighbors in clique: {len([n for n in all_neighbors if n in c_set])}\n")
        return filename

    def run_batch(self, num_runs: int):
        print(f"\n🚀 STARTING RESEARCH BATCH: {num_runs} RUNS")
        batch_start = time.time()
        successful_files = []

        for i in range(num_runs):
            run_start = time.time()
            self.current_seed = int(time.time() * 1000) % 2**32
            random.seed(self.current_seed)
            self.best_clique = []
            self.best_size = 0
            
            print(f"\n[Run {i+1}/{num_runs}] Seed: {self.current_seed}")
            
            # Define Phases
            phases = [("Early", 24, 8000, 20.0, 0.9996), 
                      ("Bridge", 26, 15000, 30.0, 0.9999), 
                      ("Final", 28, 45000, 65.0, 0.99999)]
            
            for name, target, iterations, temp, cooling in phases:
                initial = self.best_clique.copy()
                while len(initial) < target:
                    v = random.randint(0, self.n - 1)
                    if v not in initial: initial.append(v)
                
                improved, conf = self.improve_with_sa(initial, iterations, temp, cooling)
                if conf == 0:
                    self.best_size = len(improved)
                    self.best_clique = improved
                    if self.best_size >= target: continue

            run_duration = time.time() - run_start
            if len(self.best_clique) >= 28:
                fname = self.save_to_statistics(self.best_clique, self.current_seed, run_duration)
                successful_files.append(fname)
                print(f"✅ Success! Clique size 28 found in {run_duration:.2f}s")
            else:
                print(f"❌ Failed to reach 28 in this run.")

        print("\n" + "="*60)
        print(f"BATCH COMPLETE: {len(successful_files)}/{num_runs} SUCCESSFUL")
        print(f"Total Time: {time.time() - batch_start:.2f}s")
        print(f"Files saved in /statistics/ folder.")
        print("="*60)

if __name__ == "__main__":
    n = int(input("Enter number of runs to perform (e.g., 1, 10, 100): "))
    solver = Keller5ResearchBatch()
    solver.run_batch(n)

Enter number of runs to perform (e.g., 1, 10, 100):  100


Loading Keller5 data from keller_codes_d5.data...

🚀 STARTING RESEARCH BATCH: 100 RUNS

[Run 1/100] Seed: 1204383864
✅ Success! Clique size 28 found in 6.08s

[Run 2/100] Seed: 1204389964
✅ Success! Clique size 28 found in 42.00s

[Run 3/100] Seed: 1204431990
✅ Success! Clique size 28 found in 17.06s

[Run 4/100] Seed: 1204449058
✅ Success! Clique size 28 found in 84.66s

[Run 5/100] Seed: 1204533746
✅ Success! Clique size 28 found in 148.87s

[Run 6/100] Seed: 1204682636
✅ Success! Clique size 28 found in 297.09s

[Run 7/100] Seed: 1204979752
❌ Failed to reach 28 in this run.

[Run 8/100] Seed: 1205338808
✅ Success! Clique size 28 found in 26.37s

[Run 9/100] Seed: 1205365198
✅ Success! Clique size 28 found in 117.59s

[Run 10/100] Seed: 1205482803
✅ Success! Clique size 28 found in 134.35s

[Run 11/100] Seed: 1205617176
✅ Success! Clique size 28 found in 21.87s

[Run 12/100] Seed: 1205639064
✅ Success! Clique size 28 found in 292.93s

[Run 13/100] Seed: 1205932011
✅ Success! Clique s